In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from glob import glob

# ================== PARAMETRI ==================
DATA_DIR = "processed"
BATCH_SIZE = 64
EPOCHS = 15
LR = 1e-4
CHECKPOINT_EVERY = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ================== LOAD FILE LIST ==================
X_files = sorted(glob(os.path.join(DATA_DIR, "X_*.npy")))
Y_files = sorted(glob(os.path.join(DATA_DIR, "Y_*.npy")))

assert len(X_files) == len(Y_files)
days = list(zip(X_files, Y_files))

split = int(0.7 * len(days))
train_days = days[:split]
val_days = days[split:]

print(f"Train days: {len(train_days)}, Val days: {len(val_days)}")

# ================== MODEL ==================
class FallCNN(nn.Module):
    def __init__(self, window, features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 8, (5, 3), padding=(2, 1)),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Conv2d(8, 16, (5, 3), padding=(2, 1)),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            nn.Flatten(),
            nn.Linear(16 * (window // 4) * features, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

# ================== INIT MODEL SHAPE ==================
X_sample = np.load(train_days[0][0])
WINDOW, FEATURES = X_sample.shape[1], X_sample.shape[2]

model = FallCNN(WINDOW, FEATURES).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ================== TRAIN LOOP ==================
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    total = 0

    for X_path, Y_path in train_days:
        X = np.load(X_path)
        Y = np.load(Y_path)

        X = torch.tensor(X[:, None, :, :], dtype=torch.float32, device=device)
        Y = torch.tensor(Y, dtype=torch.float32, device=device).unsqueeze(1)

        perm = torch.randperm(len(X))
        for i in range(0, len(X), BATCH_SIZE):
            idx = perm[i:i + BATCH_SIZE]

            out = model(X[idx])
            loss = criterion(out, Y[idx])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * len(idx)
            total += len(idx)

        del X, Y
        torch.cuda.empty_cache()

    print(f"Epoch {epoch+1}: avg loss={epoch_loss / total:.5f}")

    if (epoch + 1) % CHECKPOINT_EVERY == 0:
        torch.save(model.state_dict(), f"fall_cnn_epoch{epoch+1}.pt")
        print(f"Saved checkpoint fall_cnn_epoch{epoch+1}.pt")

torch.save(model.state_dict(), "fall_cnn_final.pt")
print("✅ Training finished, model saved.")


Train days: 18, Val days: 9
Epoch 1: avg loss=0.00195
Epoch 2: avg loss=0.00191
Saved checkpoint fall_cnn_epoch2.pt
Epoch 3: avg loss=0.00178
Epoch 4: avg loss=0.00154
Saved checkpoint fall_cnn_epoch4.pt
Epoch 5: avg loss=0.00145
Epoch 6: avg loss=0.00126
Saved checkpoint fall_cnn_epoch6.pt
Epoch 7: avg loss=0.00119
Epoch 8: avg loss=0.00120
Saved checkpoint fall_cnn_epoch8.pt
Epoch 9: avg loss=0.00109
Epoch 10: avg loss=0.00109
Saved checkpoint fall_cnn_epoch10.pt
Epoch 11: avg loss=0.00106
Epoch 12: avg loss=0.00093
Saved checkpoint fall_cnn_epoch12.pt
Epoch 13: avg loss=0.00081
Epoch 14: avg loss=0.00077
Saved checkpoint fall_cnn_epoch14.pt
Epoch 15: avg loss=0.00111
✅ Training finished, model saved.
